In [2]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

## Imputation

In [21]:
data = pd.read_csv('data_10yrs_group_without_cbsa.csv')
data = data.drop(columns=['aqi_pm25_speciation'])

In [4]:
counties = data.county_code.unique()

In [22]:
cols = data.columns[10:]

In [29]:
med = pd.DataFrame(columns=cols)
for county, county_df in data.groupby('county_code'):
    median = []
    for col in cols:
        median.append(np.nanmedian(county_df[col]))

/Users/liyingli/anaconda3/envs/ml/lib/python3.6/site-packages/numpy/lib/function_base.py:3250: RuntimeWarning: All-NaN slice encountered
  r = func(a, **kwargs)


In [32]:
medall = []
for col in cols:
    medall.append(np.nanmedian(data[col]))

In [41]:
fillvalues = dict([(col,m) for col,m in zip(cols,medall)])

In [42]:
datafill = data.fillna(value=fillvalues)

In [65]:
labels = ['good', 'moderate', 'Unhealthy for Sensitive Groups','Unhealthy','Very Unhealthy','Hazardous','disaster']
bins = [0, 50, 100, 150, 200, 300, 500, 15000]
datafill['label'] = pd.cut(data['max_aqi_tomorrow'], bins=bins, labels=labels)

In [66]:
datafill.label.value_counts()

good                              465950
moderate                          151661
Unhealthy for Sensitive Groups     27571
Unhealthy                           5361
Very Unhealthy                       305
disaster                             141
Hazardous                             74
Name: label, dtype: int64

In [67]:
datafill.to_csv('data_fillna.csv')

In [63]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 684456 entries, 0 to 684455
Data columns (total 55 columns):
Unnamed: 0                           684456 non-null int64
county_code                          684456 non-null int64
city_name                            684456 non-null object
year                                 684456 non-null int64
month                                684456 non-null int64
day                                  684456 non-null int64
longitude                            684456 non-null float64
latitude                             684456 non-null float64
site_num                             684456 non-null int64
dayofweek                            684456 non-null int64
arithmetic_mean_co                   230708 non-null float64
arithmetic_mean_no2                  317885 non-null float64
arithmetic_mean_o3                   548028 non-null float64
arithmetic_mean_pm10                 204514 non-null float64
arithmetic_mean_pm25_frm             219586 non-nu

##  Fix the feature: dayofweek

In [ ]:
dt = datafill
dt[['year','month','day', 'dayofweek']].head(10)

In [ ]:
dt = dt.drop(columns=['dayofweek'])
dt = dt.dropna(axis=0)
dt = dt.drop(columns=['max_aqi_tomorrow'])
dt['dow'] = dt.dow.astype('uint8')
dt.to_csv('final_data.csv')